# Preprocesamiento de datos

## Ejercicios: Parte I

<div class="alert-warning">
    
Indicaciones: Los ejercicios 1, 2, 3 y 4 deben leer su data de links en Github y el código y los resultados deben ser publicados en páginas en Github.

</div>

## Ejercicio 1

<div class="alert-success">
    
1. Use two maps of points.

2. Compute the distance matrix for both maps.

3. Select one row of the distance matrix, and plot the two points with the minimal distance on top of the country of your choosing. (distancia entre punto a punto)
</div>

Mapas de puntos: Plantas de energía solar y eólica en China

Fuente: [Global Power Plant Database](https://datasets.wri.org/dataset/globalpowerplantdatabase)

In [86]:
import pandas as pd
import geopandas as gpd
import os

PowerPlants = pd.read_csv("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/data/global_power_plant_database.csv", dtype={'other_fuel3': 'object'})

China_PowerPlants = PowerPlants[PowerPlants["country"]=="CHN"]
China_PowerPlants = China_PowerPlants.loc[:,["name","gppd_idnr","capacity_mw","latitude","longitude","primary_fuel"]]
China_PowerPlants.reset_index(drop=True, inplace=True)

#Dataframe a GeoDataframe:
China_PowerPlants_gpd = gpd.GeoDataFrame(data=China_PowerPlants.copy(),
                                        geometry=gpd.points_from_xy(x=China_PowerPlants.longitude,
                                        y=China_PowerPlants.latitude),
                                        crs=4326)

#Cambiando los CRSs por unos proyectados y que coincidan con los de China3415:
China3415_PowerPlants = China_PowerPlants_gpd.to_crs(3415)

#Mapas de puntos: Plantas de energía solar y eólica
China3415_SolarPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Solar"]
China3415_SolarPowerPlants = China3415_SolarPowerPlants.rename(columns={"name":"SolarPowerPlant_Name"})
China3415_SolarPowerPlants = China3415_SolarPowerPlants.reset_index()
China3415_SolarPowerPlants.drop(columns="index",inplace=True)


China3415_WindPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Wind"]
China3415_WindPowerPlants = China3415_WindPowerPlants.rename(columns={"name":"WindPowerPlant_Name"})
China3415_WindPowerPlants = China3415_WindPowerPlants.reset_index()
China3415_WindPowerPlants.drop(columns="index",inplace=True)


# Geopackage con los 2 mapas de puntos:
China3415_SolarPowerPlants.to_file(os.path.join("maps","China_3415","China3415_PowerPlants.gpkg"), layer="SolarPower_Plants", driver="GPKG",index="")
China3415_WindPowerPlants.to_file(os.path.join("maps","China_3415","China3415_PowerPlants.gpkg"), layer="WindPower_Plants", driver="GPKG",index="")

## Ejercicio 2

<div class="alert-success">
    
1. Use a map of points and a map of lines.

2. Compute the distance matrix for both maps.

3. Select one line of the distance matrix, and plot the closests and the farthest point to that line. (Se puede seleccionar un río o un sistema y calcular el punto más cercano o lejano a la línea. Si se escoge un punto, se calcula la línea más cercana y lejana)
    
    
</div>

Plantas de energía nuclear en China:

In [87]:
China3415_NuclearPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Nuclear"]
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.rename(columns={"name":"NuclearPowerPlant_Name"})
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.reset_index()
China3415_NuclearPowerPlants.drop(columns="index",inplace=True)

China3415_NuclearPowerPlants.to_file(os.path.join("maps","China_3415","China3415_PowerPlants.gpkg"), layer="NuclearPower_Plants", driver="GPKG",index="")

- Mapa de China: [World_Countries](https://www.efrainmaps.es/english-version/free-downloads/world/)

- Ríos en China: [World_Hydrography](https://www.efrainmaps.es/english-version/free-downloads/world/)

In [120]:
#Mapa de China
countries = gpd.read_file("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_Countries/World_Countries.shp")
China = countries[countries.COUNTRY=='China']
China3415_country = China.to_crs(3415)
China3415_country = China3415_country.reset_index()
China3415_country.drop(columns="index",inplace=True)
China3415_country.to_file(os.path.join("maps","China_3415","China3415_maps.gpkg"), layer="Country", driver="GPKG",index="")

#Mapa de ríos en el mundo
Riverslink = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_Hydrography/World_Hydrography.shp"
Rivers = gpd.read_file(Riverslink)
Rivers3415 = Rivers.to_crs(3415)

#Recortar los ríos para que solo se encuentren los que encierran a China
China3415_Rivers = gpd.clip(gdf = Rivers3415, mask = China3415_country)
China3415_Rivers = China3415_Rivers.reset_index()
China3415_Rivers.drop(columns="index",inplace=True)
China3415_Rivers = China3415_Rivers.rename(columns={"NAME":"River_Name"})
China3415_Rivers = China3415_Rivers.rename(columns={"NAME":"River_Name", "SYSTEM":"System"})
China3415_Rivers.replace({None:"None"}, inplace = True)

China3415_Rivers.to_file(os.path.join("maps","China_3415","China3415_maps.gpkg"), layer="Rivers", driver="GPKG",index="")

## Ejercicio 3

<div class="alert-success">
    
1. Create a set of points and a set of polygons

2. Compute the distance matrix for both sets.

3. Select one polygon of the distance matrix, and plot the closests and the farthest point to that polygon.
</div>   

Conjunto de polígonos: Convex hulls de ríos acorde al sistema al que pertenecen. Los que no forman parte de un sistema de ríos, se han engoblado en un mismo convex hull.

In [119]:
Lakeslink = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/Natural_Earth_lakes/ne_110m_lakes.shp"
Lakes = gpd.read_file(Lakeslink)
Lakes3415 = Lakes.to_crs(3415)
Lakes3415
#China3415_Lakes = gpd.clip(gdf = Lakes3415, mask = China3415_country)
#China3415_Lakes

,scalerank,featurecla,name,name_alt,admin,min_zoom,min_label,name_abb,label,wikidataid,...,name_tr,name_vi,name_zh,ne_id,name_fa,name_he,name_uk,name_ur,name_zht,geometry
0,0,Lake,Lake Baikal,None,None,1.0,3.0,L. Baikal,Lake Baikal,Q5513,...,Baykal,Baikal,贝加尔湖,1159113127,دریاچه بایکال,ימת באיקל,Байкал,جھیل بیکال,贝加尔湖,"POLYGON ((-96835.855 4247909.614, -99182.442 4..."
1,0,Lake,Lake Winnipeg,None,None,1.0,3.0,L. Winnipeg,Lake Winnipeg,Q3272,...,Winnipeg,Winnipeg,溫尼伯湖,1159106747,دریاچه وینیپگ,אגם ויניפג,Вінніпег,جھیل ونیپیگ,溫尼伯湖,"POLYGON ((10611011.338 9395573.922, 10614856.1..."
2,0,Lake,Great Slave Lake,None,None,1.0,3.0,Great Slave L.,Great Slave Lake,Q5539,...,Büyük Esir,Slave Lớn,大奴湖,1159106729,دریاچه گریت اسلیو,ימת העבדים הגדולה,Велике Невільниче озеро,گریٹ سلیو جھیل,大奴湖,"POLYGON ((8945617.245 9207878.382, 8841715.303..."
3,0,Lake,Lake Ontario,Great Lakes,admin-0,1.0,3.0,L. Ontario,Lake Ontario,Q1062,...,Ontario,Ontario,安大略湖,1159106765,دریاچه انتاریو,ימת אונטריו,Онтаріо,جھیل انٹاریو,安大略湖,"POLYGON ((12659324.153 10031117.579, 12651185...."
4,0,Lake,Lake Erie,Great Lakes,admin-0,1.0,3.0,L. Erie,Lake Erie,Q5492,...,Erie,Erie,伊利湖,1159106757,دریاچه ایری,ימת אירי,Ері,جھیل ایری,伊利湖,"POLYGON ((12595678.351 9650509.226, 12627574.8..."
5,0,Lake,Lake Superior,Great Lakes,admin-0,1.0,3.0,L. Superior,Lake Superior,Q1066,...,Superior,Thượng,苏必利尔湖,1159112991,دریاچه سوپریور,ימת סופיריור,Озеро Верхнє,جھیل سپیریئر,苏必利尔湖,"POLYGON ((11682315.444 9571024.563, 11659990.0..."
6,0,Lake,Lake Victoria,None,admin-0,1.0,3.0,L. Victoria,Lake Victoria,Q5505,...,Victoria,Victoria,維多利亞湖,1159113191,دریاچه ویکتوریا,אגם ויקטוריה,Вікторія,جھیل وکٹوریہ,維多利亞湖,"POLYGON ((-8610260.742 477305.075, -8610260.74..."
7,0,Lake,Lake Ladoga,None,None,1.0,3.0,L. Ladoga,Lake Ladoga,Q15288,...,Ladoga,Ladoga,拉多加湖,1159113095,دریاچه لادوگا,ימת לדוגה,Ладозьке озеро,جھیل لاڈوگا,拉多加湖,"POLYGON ((-5364826.097 7005692.908, -5364826.0..."
8,0,Lake,Lake Balkhash,Balqash Köli,None,1.0,3.0,L. Balkhash,Lake Balkhash,Q134485,...,Balkaş,Balkhash,巴尔喀什湖,1159113111,دریاچه بالخاش,ימת בלחש,Балхаш,جھیل بالخاش,巴爾喀什湖,"POLYGON ((-2463042.952 3784243.389, -2463042.9..."
9,0,Lake,Lake Tanganyika,None,admin-0,1.0,3.0,L. Tanganyika,Lake Tanganyika,Q5511,...,Tanganika,Tanganyika,坦干依喀湖,1159113185,دریاچه تانگانیکا,אגם טנגניקה,Танганьїка,جھیل ٹانگانیکا,坦干依喀湖,"POLYGON ((-9451248.592 -263776.349, -9483292.9..."


In [114]:
China3415_Railroads

,F_CODE_DES,HYC_DESCRI,NAM,ISO,NAME_0,geometry
0,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (-1878374.535 3827862.360, -1878844..."
1,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (-1879932.387 3827482.277, -1880821..."
2,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (-1880821.221 3828258.916, -1881522..."
3,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (-1882744.882 3826479.784, -1882346..."
4,River/Stream,Perennial/Permanent,ERTIX HE,CHN,China,"LINESTRING (-1806513.156 3796172.219, -1806148..."
...,...,...,...,...,...,...
51427,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (-2438778.122 3293683.567, -2438778..."
51428,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (-2444420.972 3291433.375, -2444420..."
51429,River/Stream,Perennial/Permanent,BAYANKOL,CHN,China,"MULTILINESTRING ((-2453998.290 3293480.239, -2..."
51430,River/Stream,Perennial/Permanent,UNK,CHN,China,"LINESTRING (2005362.026 3220876.644, 2005361.9..."


In [11]:
#GeoDataFrame, agrupando por ríos
China3415_RiverSystems = China3415_MajorRivers.dissolve(by="System")
China3415_RiverSystems = China3415_RiverSystems.drop(columns = ["River_Name","MILES","KILOMETERS"])
China3415_RiverSystems = China3415_RiverSystems.reset_index()
China3415_RiverSystems.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer='RiverSystems', driver="GPKG",index="")

#Convertir a convex hull y luego a un GeoDataFrame con polígonos
China3415_RiverSystems_hulls = China3415_RiverSystems.convex_hull.to_frame()
China3415_RiverSystems_hulls.rename(columns = {0:"geometry"}, inplace = True)
China3415_RiverSystems_hulls = China3415_RiverSystems_hulls.set_geometry("geometry")
China3415_RiverSystems_hulls = China3415_RiverSystems.reset_index()
China3415_RiverSystems_hulls

China3415_RiverSystems_hulls.to_file(os.path.join("maps","China3415","China3415_Maps.gpkg"), layer='RiverSystems_hulls', driver="GPKG",index="")

Conjunto de puntos: Plantas de energía nuclear en China

## Ejercicio 4

<div class="alert-success">
    
1. Create a set of points and a set of lines

2. Get the buffer for the lines, select different values for the distance. Jugar con varias distancias, pero solo colocar una que tenga resultados interesantes en el notebook final.

3. Keep the points that are within the buffer (as in point 2, you need to play with differn distances until you show something interesting. Todos los puntos que están dentro del buffer creado para la línea.
   

</div>   

- Set de puntos: China3415_NuclearPowerPlants
- Set de líneas: China3415_RiverSystems